In [2]:
from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

/Users/chidam/anaconda/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
df1=pd.read_csv('sentiment_training.csv', sep=',', header=None)

In [5]:
df1.head()

,0,1
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...


In [8]:
df1['rating']=df1[0]

In [9]:
df1['review']=df1[1]

In [10]:
df1.drop([0, 1], axis=1)

,rating,review
0,1,The Da Vinci Code book is just awesome.
1,1,this was the first clive cussler i've ever rea...
2,1,i liked the Da Vinci Code a lot.
3,1,i liked the Da Vinci Code a lot.
4,1,I liked the Da Vinci Code but it ultimatly did...
5,1,that's not even an exaggeration ) and at midni...
6,1,"I loved the Da Vinci Code, but now I want some..."
7,1,"i thought da vinci code was great, same with k..."
8,1,The Da Vinci Code is actually a good movie...
9,1,I thought the Da Vinci Code was a pretty good ...


In [11]:
(X_train, y_train)=df1['review'].tolist(), df1['rating'].tolist()

In [12]:
df2=pd.read_csv('sentiment_testdata.csv', sep=',', header=None)

In [13]:
df2['review']=df2[0]
df2.drop([0], axis=1)

,review
0,"I don't care what anyone says, I like Hillary..."
1,"harvard is dumb, i mean they really have to be..."
2,I'm loving Shanghai > > > ^ _ ^.
3,harvard is for dumb people.
4,"As i stepped out of my beautiful Toyota, i hea..."
5,"Bodies being dismembered, blown apart, and mut..."
6,I love Harvard Square in the fall.
7,London = amazing...
8,I HATE LONDON!..
9,I love MIT so much...


In [14]:
X_test=df2['review'].tolist()

In [15]:
# truncate and pad input sequences
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
numpy.random.seed(7)

from __future__ import print_function

import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model

MAX_SEQUENCE_LENGTH = 700
MAX_NUM_WORDS = 10000
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_train)
sequences1 = tokenizer.texts_to_sequences(X_train)

tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(X_test)
sequences2 = tokenizer.texts_to_sequences(X_test)



X_train_fair = sequence.pad_sequences(sequences1, maxlen=MAX_SEQUENCE_LENGTH)
X_test_fair = sequence.pad_sequences(sequences2, maxlen=MAX_SEQUENCE_LENGTH)




In [36]:
# create the model
model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
model.fit(X_train_fair, y_train, epochs=3, batch_size=64, verbose=2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 700, 100)          1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
 - 340s - loss: 0.3207 - acc: 0.8692
Epoch 2/3
 - 340s - loss: 0.0555 - acc: 0.9827
Epoch 3/3
 - 415s - loss: 0.0147 - acc: 0.9962


In [37]:
yhat = model.predict_classes(X_test_fair, verbose=0)

In [38]:
yhat

array([[0],
       [0],
       [1],
       ...,
       [1],
       [0],
       [0]], dtype=int32)

In [ ]:
#tuning the hyperparameters and trying Bidirectional(LSTM): trying the one from above with epochs=10
from keras.regularizers import l2 # L2-regularisation
from keras.layers import Bidirectional, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping
#from keras.layers.normalization import BatchNormalization # batch normalisation
#inp_norm = BatchNormalization()(inp) 

#conv_1 = BatchNormalization()(conv_1)
l2 = 0.0001
model2 = Sequential()
num_epochs=20

model2.add(Embedding(MAX_NUM_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))

model2.add(Bidirectional(LSTM(100)))
model2.add(Dropout(0.2))
model2.add(Dense(3, activation='sigmoid', kernel_regularizer=regularizers.l2(0.01)))
model2.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['acc'])
print(model2.summary())
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)
#model2.fit(X_train_fair, y_train, epochs=10, batch_size=128, verbose=0)

es=EarlyStopping(monitor='loss', patience=5, verbose=0, mode='auto')
model2.fit(X_train_fair, y_train, batch_size=128, epochs=20, verbose=0, callbacks=[es], validation_data=None, shuffle=True, class_weight=None, sample_weight=None)

#model2.fit_generator((X_train_fair, Y_train, batch_size=128), steps_per_epoch=X_train.shape[0], epochs=num_epochs, verbose=0, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 700, 100)          1000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               160800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 603       
Total params: 1,161,403
Trainable params: 1,161,403
Non-trainable params: 0
_________________________________________________________________
None
